O objetivo deste notebook é explorar alguns aspectos específicos da pesquisa realizada pelo Kaggle sobre o status do data science em 2017 relacionados ao Brasil. O Kaggle publicou uma analise/relatório interessante sobre a pesquisa, que pode ser acessado [neste link](https://www.kaggle.com/surveys/2017). Recomendo aos interessados interagir com a analise citada, por exemplo filtrando os resultados do Brasil. Porém, este relatório publicado não cobre alguns detalhes que pretendo explorar aqui de forma incremental e gradual.

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('../input/multipleChoiceResponses.csv', encoding="ISO-8859-1", low_memory=False)
br_data = data[data.Country == 'Brazil']

Com os dados importados, o filtro é feito na segunda linha onde selecionamos apenas as respostas onde o campo Country é igual ao Brasil. Vamos ver o tamanho deste subset.

## Sobre as Pessoas

In [ ]:
print("%s brasileiros responderam a pesquisa do Kaggle" % br_data.shape[0])

In [ ]:
counts = data.Country.value_counts().head(10)
counts.plot(kind='pie', figsize=(8,8), autopct='%1.1f%%');
plt.legend(labels=counts.values, loc="best");

O Brasil é o 7º país de origem com mais respostas na pesquisa com 4,1% das respostas!

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)

brgender = br_data.GenderSelect.value_counts()
brgender.plot(kind='pie', figsize=(15,7), autopct='%1.1f%%', ax=axes[0]);

gender = data[data.Country != 'Brazil'].GenderSelect.value_counts()
gender.plot(kind='pie', figsize=(15,7), autopct='%1.1f%%', ax=axes[1]);

No gráfico acima vemos como esta área ainda é pouco inclusiva, apenas 8.2% das respostas brasileiras foram feitas por pessoas que se identificam com o sexo feminino, enquanto na média dos outros países esse número é o dobro, 17% de participantes femininos.

Mesmo os 17% da média (sem o Brasil) ainda é um número muito pequeno de participação feminina para qualquer área.

In [ ]:
br_data.Age.value_counts(bins=10).plot(kind='bar', figsize=(15,6), rot=40, legend=True);

No gráfico acima vemos a distribuição dos participantes por idade, ficando a maioria concentrados entre 26 e 32 anos.

## Sobre o Trabalho

In [ ]:
employ = br_data.EmploymentStatus.value_counts()
employ.plot(kind='pie', figsize=(7,7), autopct='%1.1f%%');
plt.legend(labels=employ.values, loc="best");

Acima vemos que a maior partes dos participantes estão contratados em regime de tempo integral, mas que existem 10% de profissionais procurando emprego e outros 5% que ainda não estão buscando uma colocação.

Obs: Seria interessante avaliar mais de perto estes profissionais, formação, experiência, idade e etc.

In [ ]:
jobtitle = br_data.CurrentJobTitleSelect.value_counts()
jobtitle.plot(kind='bar', figsize=(15,6), rot=40, legend=True);

In [ ]:
sns.factorplot(y='CurrentJobTitleSelect', hue='TitleFit', kind='count', data=br_data, size=8);

Nos dois gráficos acima vemos que a maioria dos participantes recebe/usa o título "cientista de dados" mas que o mesmo concentra o maior número de "não concordância" (TitleFit = Poorly).

Ao mesmo tempo percebemos que poucas pessoas recebem/usam os títulos de "engenheiro de machine learning" ou "analista de dados". Acredito que estes dados corroboram uma percepção pessoal que, por sua vez, confirma uma oportunidade. 

Minha percepção é que estamos alguns anos "atrás" do mercado externo (América do Norte e Europa) nesta e em outras áreas, o que leva o mercado interno a estar iniciando enquanto o mercado externo já esta consolidado neste campo. E não poderia ser diferente, uma vez que estas tecnologias são criadas e desenvolvidas lá e não aqui.

Por outro lado, existe a oportunidade de antecipar os movimentos do mercado apenas observando o que acontece fora do Brasil (America do Norte e Europa). Lá, cargos como engenheiro de machine learning, analista de dados e engenheiro de dados são comuns enquanto aqui estão apenas começando a surgir. Isso nos dá tempo para nos prepararmos para uma demanda quase certa.

Obs: Analisar a mesma relação entre "CurrentJobTitleSelect" e "TitleFit" no mercado externo

## Sobre a Remuneração

In [ ]:
salary = br_data[br_data.CompensationAmount.notnull()]
print('%s pessoas informaram seus salários no subset Brasil.' % salary.shape[0])

In [ ]:
salary.loc[:, 'CompensationAmount'] = salary.CompensationAmount.apply(
    lambda x: 0 if (pd.isnull(x) or (x=='-') or (x==0)) else float(x.replace(',',''))
)

In [ ]:
def monthly2yearly(salary):
    if salary < 30_000:
        return salary * 12
    return salary

salary = salary[(salary.CompensationAmount < 1_000_000) & (salary.CompensationAmount > 2_000)]
salary.loc[:, 'CompensationAmount'] = salary.CompensationAmount.apply(monthly2yearly)

In [ ]:
gender_salary = salary.groupby('GenderSelect').agg({'CompensationAmount':['mean','std']})
gender_salary.plot(kind='bar', figsize=(8,6));

No gráfico acima vemos a confirmação de uma realidade que todos já conhecemos, ainda remuneramos menos mulheres nos mesmos cargos que homens em média. A mesma tendência pode ser observada nos dados gerais publicados pelo relatório do Kaggle (citado no começo deste documento) na questão "What is your full-time annual salary?".

In [ ]:
group = salary.groupby('Age')['CompensationAmount'].mean()
group = group.reset_index()
age_salary = group.groupby(pd.cut(group['Age'], np.array([20, 30, 40, 50, 60, 80]))).mean()
age_salary['CompensationAmount'].plot(kind='bar', figsize=(8,6), legend=True);

Acima vemos que a concentração das melhores remunerações no Brasil são pagas para pessoas entre 40 e 50 anos. Aqui acredito que exista alguma distorção devido aos poucos dados disponíveis (~150 pessoas informaram seus salários) mas não posso afirmar com certeza.

In [ ]:
edu_salary = salary.groupby('FormalEducation').agg({'CompensationAmount': ['mean', 'std']})
edu_salary.plot(kind='barh', figsize=(8, 7)).legend(bbox_to_anchor=(-.3, .1));

O mesmo resultado surpreendente acontece no gráfico acima, que indica que as melhores remunerações são pagas a pessoas sem um diploma universitário. Posso estar sendo tendencioso, mas acredito que estes dados não condizem com a realidade.